## OWASP ZAP Mini Workshop - ZAPv2.7.0

#### Abhay Bhargav - we45

#### Objective
Over the last 2 years, I have worked with and implemented several SAST and DAST tools as part of a Continuous Delivery Pipeline. In this time, I have seen that there are few tools that match the flexibility and capabilities of OWASP's Zed Attack Proxy, especially from the perspective of integrating into a Continuous Delivery Pipeline. 

As part of training programs that I have delivered at OWASP AppSecUSA, EU and DEFCON, I have trained several folks on OWASP ZAP's API and Scripting features. This is a shorter version of the same training program that I have delivered in these places. 

The objective of this is as follows: 
- To get someone quickly up and running with ZAP and its feature-rich API and Scripting Features
- To NOT use presentations. It takes FOREVER prepping presentation. Writing everything in Markdown and Python is far easier
- To cover essential functionality that I think would be useful to folks. I am sure I will be missing some cool features here. Please bear with me
- To reference some other useful tools and libraries that I have been working with, either ZAP related or otherwise. 

#### Requirements
- Python 3.6.1
- Please install all requirements from `requirements.txt`. This includes ZAP's Python API
- Download and Install ZAP 2.7.0 (Download from https://github.com/zaproxy/zaproxy/wiki/Downloads)

#### Warnings
This code is given to you as is, where is. I don't take any responsibility for how/where you use it. This is NOT production ready in any way.

#### Thanks
- I would like to thank Simon Bennetts (@psiinon) and the rest of the OWASP ZAP Development Team. They do a truly admirable job of maintaining and managing ZAP for all of us. 
- I would like to thank a few members of my amazing team! - Nithin, Sharath and Tilak. Thanks for all that you do

#### Other Useful Projects
- ZAP Robot Framework Integration - https://bitbucket.org/we45abhay/robozap
- Vulnerable App used in this exercise - https://bitbucket.org/we45abhay/defcon_vul_app/

###### Let's get started.....


### Pre-start Setup Instructions

- Download and Install ZAP
- Install some plugins from the ZAP Marketplace, namely: 
    - Python Scripting
    - Export Report
- You may choose to install several other plugins. However, for this training, you only need these plugins
- Open ZAP > Tools > Options > API 
    - Make sure that it's enabled, for UI as well
    ![ZAP API Screen](images/api_zap.jpg)
    - I have disabled API key in this case. You can enable if you would like. However, remember that you need the API Key for every call to ZAP's API if you enable the API Key.
    

### ZAP 2.7.0 API

If you enable the UI Option in ZAP's API Options screen (as done above), you will have access to ZAP's API directly over the browser. You will typically have ZAP running on port 8080 of your localhost. In my case, I have it running on port 8090

The API is available only when you have started ZAP. 

![ZAP API Screen](images/api_screen.jpg)

Very soon, we are going to start zap using ZAP's CLI and get it up and running. This is very useful when you want to start ZAP without using the UI. 

In [1]:
# we will be using python's subprocess to start ZAP in GUI and headless modes

import subprocess
import os
from IPython.display import display

#GUI ZAP
base_path = '/Applications/OWASP_ZAP.app/Contents/Java/'
gui_command = base_path + 'zap.sh -config api.disablekey=true -port 8090'
# you can use the config param to specify set specific configurations you need when you launch the CLI.
# In this case, I am (actually don't need to) starting ZAP with the API Key disabled and listening port 8090

headless_command = base_path + 'zap.sh -daemon -config api.disablekey=true -port 8090'
#by specifying 'daemon' in the CLI, ZAP starts in Headless mode

zap_process = subprocess.Popen(gui_command.split(' '), stdout = open(os.devnull, 'w'))

If you have run the code above correctly, you should see that ZAP is now running in GUI Mode. We will be using GUI Mode for this workshop, so you can easily see everything going on within ZAP.

Now that we have ZAP open and running, let's use its API to perform some scans. I will be using it's Python API to perform this. However, ZAP can be used with Java, JavaScript, PHP, Ruby and directly as a REST API

##### Links: 
- NPM: https://www.npmjs.com/package/zaproxy
- Java: https://github.com/zaproxy/zaproxy/releases
- Ruby: https://github.com/vpereira/owasp_zap

Let's start with running the spider against intentionally vulnerable site `demo.testfire.net`

In [2]:
from zapv2 import ZAPv2 as ZAP #import ZAP library
import time

zap = ZAP(proxies = {'http': 'http://localhost:8090', 'https': 'http://localhost:8090'})
#setting the local ZAP instance that is open on your local system

In [2]:
target_site = 'http://demo.testfire.net'

zap.urlopen(target_site)
#opens up the the target site. Makes a single GET request

spider_id = zap.spider.scan(target_site)
#this line of code kicks off the ZAP Default Spider. This returns an ID value for the spider

print("Spider ID for the initiated spider scan is: {0}".format(spider_id))


#now we can start monitoring the spider's status
while int(zap.spider.status(spider_id)) < 100:
    print("Current Status of ZAP Spider: {0}%".format(zap.spider.status(spider_id)))
    time.sleep(4)

Spider ID for the initiated spider scan is: 0
Current Status of ZAP Spider: 0%
Current Status of ZAP Spider: 33%
Current Status of ZAP Spider: 38%
Current Status of ZAP Spider: 61%
Current Status of ZAP Spider: 73%
Current Status of ZAP Spider: 97%


If everything went well, you should see that the spider ran successfully, and identified a bunch of in-scope and Out of scope URLs and params. 

Before we proceed with scans and other "vulnerability discovery" activities. Let's get some information about the target site from ZAP's API

#### Enumerating the Target Site

##### Information about the app's content - List of URLs

In [3]:
#fetch list of urls enumerated by ZAP
zap.core.urls()[:10]
#fetch upto 10 results (to fit on screen). You can remove the "[:10]" to fetch all results

['http://demo.testfire.net',
 'http://demo.testfire.net/admin',
 'http://demo.testfire.net/admin/clients.xls',
 'http://demo.testfire.net/bank',
 'http://demo.testfire.net/bank/login.aspx',
 'http://demo.testfire.net/bank/login.aspx',
 'http://demo.testfire.net/cgi.exe',
 'http://demo.testfire.net/comment.aspx',
 'http://demo.testfire.net/default.aspx',
 'http://demo.testfire.net/default.aspx?content=jobs/20061024.htm']

Let's fetch some content types from the site with ZAP's API

In [4]:
display(zap.stats.site_stats(site = target_site)[:10])
#fetch upto 10 results (to fit on screen). You can remove the "[:10]" to fetch all results

[{'stats.code.200': 97,
  'stats.code.302': 2,
  'stats.code.404': 4,
  'stats.code.500': 2,
  'stats.contentType.application/octet-stream': 2,
  'stats.contentType.application/pdf': 2,
  'stats.contentType.application/vnd.ms-excel': 2,
  'stats.contentType.image/gif': 8,
  'stats.contentType.image/jpeg': 25,
  'stats.contentType.text/css': 2,
  'stats.contentType.text/html': 9,
  'stats.contentType.text/html; charset=utf-8': 57,
  'stats.contentType.text/plain': 2,
  'stats.contentType.text/xml': 2,
  'stats.responseTime.1024': 18,
  'stats.responseTime.256': 9,
  'stats.responseTime.4096': 3,
  'stats.responseTime.512': 75,
  'stats.tag.Comment': 12,
  'stats.tag.Form': 59,
  'stats.tag.Hidden': 2,
  'stats.tag.Object': 2,
  'stats.tag.Password': 3,
  'stats.tag.Script': 6,
  'stats.tag.SetCookie': 3}]

ZAP has a very useful list of params tested in the application. This can be accessed as follows....

In [5]:
zap.params.params()[0]['Parameter'][:5]
#fetch upto 5 results (to fit on screen). You can remove the "[0]['Parameter'][:5]" to fetch all results

[{'name': 'ASP.NET_SessionId',
  'site': 'demo.testfire.net:80',
  'timesUsed': '99',
  'type': 'cookie'},
 {'Flags': ['session', 'path=/', 'HttpOnly']},
 {'Values': ['s05zue3wcrmaloqmyedfhi55', 'scmslk22ta3tvo5501dcpg45']},
 {'name': 'amSessionId',
  'site': 'demo.testfire.net:80',
  'timesUsed': '99',
  'type': 'cookie'},
 {'Flags': ['path=/']}]

Now, I am not sure if you noticed, but ZAP automatically starts gathering some details about vulnerabilities in the site using its `Passive Scan` Feature. This is very useful to silently enumerate vulnerabilites related to HTTP Headers, Session Tokens, etc without actually firing attack packets at the target.

#### Let's look at the stuf that ZAP Scans for passively...

In [5]:
display(zap.pscan.scanners[0:5])
#fetch upto 5 results (to fit on screen). You can remove the "[0:5]" to fetch all results

[{'alertThreshold': 'DEFAULT',
  'enabled': 'true',
  'id': '50001',
  'name': 'Script Passive Scan Rules',
  'quality': 'release'},
 {'alertThreshold': 'DEFAULT',
  'enabled': 'true',
  'id': '50003',
  'name': 'Stats Passive Scan Rule',
  'quality': 'release'},
 {'alertThreshold': 'DEFAULT',
  'enabled': 'true',
  'id': '90022',
  'name': 'Application Error Disclosure',
  'quality': 'release'},
 {'alertThreshold': 'DEFAULT',
  'enabled': 'true',
  'id': '10015',
  'name': 'Incomplete or No Cache-control and Pragma HTTP Header Set',
  'quality': 'release'},
 {'alertThreshold': 'DEFAULT',
  'enabled': 'true',
  'id': '10019',
  'name': 'Content-Type Header Missing',
  'quality': 'release'}]

As you can see from the above output, some of these are enabled and some are disabled. The number of findings from the Passive Scan are correlated with the number of URls that are crawled by ZAP. Before we go deep into vulnerability scanning, you can actually see the existing vulnerabilities that have been identified by ZAP with this...

In [6]:
# get an existing list of vulnerabilities
zap.core.alerts(baseurl=target_site)[:2]
#fetch upto 2 results (to fit on screen). You can remove the "[:2]" to fetch all results

[{'alert': 'Cookie No HttpOnly Flag',
  'attack': '',
  'confidence': 'Medium',
  'cweid': '16',
  'description': 'A cookie has been set without the HttpOnly flag, which means that the cookie can be accessed by JavaScript. If a malicious script can be run on this page then the cookie will be accessible and can be transmitted to another site. If this is a session cookie then session hijacking may be possible.',
  'evidence': 'Set-Cookie: amSessionId',
  'id': '0',
  'messageId': '1',
  'method': 'GET',
  'name': 'Cookie No HttpOnly Flag',
  'other': '',
  'param': 'amSessionId',
  'pluginId': '10010',
  'reference': 'http://www.owasp.org/index.php/HttpOnly',
  'risk': 'Low',
  'solution': 'Ensure that the HttpOnly flag is set for all cookies.',
  'sourceid': '3',
  'url': 'http://demo.testfire.net/',
  'wascid': '13'},
 {'alert': 'Web Browser XSS Protection Not Enabled',
  'attack': '',
  'confidence': 'Medium',
  'cweid': '933',
  'description': "Web Browser XSS Protection is not enabl

#### Customizing Scan Policy

One of the things you might have noticed with DAST Scanning, is that with fully-enabled scan policies, it takes a significantly long time to perform the scan. Let's create a lightweight scan policy through ZAP's API. This is a new API endpoint in ZAP 2.7.0

In [9]:
#creating a new scan policy with attack threshold and attack strength
light = zap.ascan.add_scan_policy('Light', alertthreshold='Low', attackstrength='Low')

If everything has gone correctly, a new scan policy should have been created with the title "Light". Let's fetch all the scan policies in our ZAP instance

In [10]:
# querying all scan policies by name
zap.ascan.scan_policy_names

['Default Policy', 'Light']

## Let's start scanning!!

If you see "Light" the previous cell, it means that you are set and ready to scan. Let's get started....

In [7]:
# using ZAP's ascan object to start scanning, with the "Light" Policy. If you don't specify the policy
# ZAP Automatically uses the "Default" policy
active_scan_id = zap.ascan.scan(target_site, scanpolicyname='Light')

print("active scan id: {0}".format(active_scan_id))

#now we can start monitoring the spider's status
while int(zap.ascan.status(active_scan_id)) < 100:
    print("Current Status of ZAP Active Scan: {0}%".format(zap.ascan.status(active_scan_id)))
    time.sleep(10)

active scan id: 0
Current Status of ZAP Active Scan: 0%
Current Status of ZAP Active Scan: 0%
Current Status of ZAP Active Scan: 18%
Current Status of ZAP Active Scan: 18%


### Pulling results to report

The next thing that we need after scanning is to pull results to generate some reports. ZAP provides an XML and HTML Report by default. However, with the "Export Report" plugin, ZAP allows you to export the XHTML, PDF and JSON variants of the report, replete with HTTP Requests, Responses and detailed information about the vulnerability. This can also be customized during the fetch, and done through the API

** Note: ZAP's Export Report functionality cannot be used through it's Python API. We will be using it's REST API for this API endpoint. Please refer to https://github.com/zaproxy/zap-extensions/wiki/HelpAddonsExportreportExportreport for additional details about ZAP's Export Report**

In [18]:
import requests #since we will be making a REST API Call

url = 'http://localhost:8090/JSON/exportreport/action/generate/'
export_path = os.getcwd() + '/testfire.json'
extension = 'json'
source_info = 'Vulnerability Report for Demo Testfire;Abhay Bhargav;TestfireTeam;January 8 2018;January 8 2018;v1;v1;Testfire Scan Report'
alert_severity = 't;t;t;t' #High;Medium;Low;Info
alert_details = 't;t;t;t;t;t;f;f;f;f' #CWEID;#WASCID;Description;Other Info;Solution;Reference;Request Header;Response Header;Request Body;Response Body
data = {'absolutePath': export_path, 'fileExtension': extension, 'sourceDetails': source_info, 'alertSeverity': alert_severity, 'alertDetails': alert_details}

r = requests.post(url, data = data)
print(r.content)

b'{"Result":"OK"}'


If all's well, then you should go back to the directory in which you have this Notebook and find that there's a file called `testfire.json`. If you open `testfire.json` with a text editor or browser, you should see a detailed JSON report with the params that we have provided. 

This is great! We have now run some tests with ZAP with unauthenticated scans. But that's not necessarily the best thing. Most of us don't need to run ZAP this way. We ideally want to run "authenticated scans" for higher coverage of our security testing efforts. So...

### Let's run an authenticated scan against a webservice. 
- To do this correctly, we are going to stop ZAP first
- Start it again
- Run the parameterization
- Run the scan

#### Stop ZAP

In [8]:
zap.core.shutdown() #shuts down the ZAP Scanner

'OK'

#### Start ZAP Again
If everything is has gone well with the above process. You can start ZAP like you did before. 

In [9]:
zap_process = subprocess.Popen(gui_command.split(' '), stdout = open(os.devnull, 'w'))

#### Running Parameterization

##### Starting Vulnerable App Docker Container
You can run completely automated authenticated scans in the following ways

Using, Selenium: 
![ZAP with Selenium](images/zap_selenium.jpg)

With ZAP "Pre-Canned" Sessions
![ZAP Sessions](images/zap_session.jpg)

With ZAP's Zest Scripts
![ZAP Zest Scripts](images/zap_zest.png)

with Selenium, in case you are testing a browser-based web application, or in this case, a simple HTTP Client, since we are testing a web service. The most important thing to remember is that you need to proxy all the traffic through OWASP ZAP, so ZAP can use this as an input to perform the active scan with authentication. 

We will be testing a Flask based web service that requires a JSON Web Token (JWT) for Authenticated Operations

Let's open up a Terminal Window and start our Vulnerable Flask Application Docker Container with this command: 

`docker run -p 5050:5050 abhaybhargav/vul_flask`

If all goes well, you should see something like this: 

```
/usr/local/lib/python2.7/site-packages/flask_sqlalchemy/__init__.py:839: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
```

##### Parameterization - Web Service

Let's authenticate and perform some actions on this web Service

In [7]:
import requests

target_url = 'http://localhost:5050'
proxies = {
  'http': 'http://127.0.0.1:8090',
  'https': 'http://127.0.0.1:8090',
} #passing all traffic through OWASP ZAP

auth_dict = {'username': 'admin', 'password': 'admin123'}

login = requests.post(target_url + '/login', proxies = proxies, json=auth_dict)

if login.status_code == 200: #if login is successful
    auth_token = login.headers['Authorization']
    auth_header = {"Authorization": auth_token}
    #now lets run some operations
    # GET Customer by ID
    
    get_cust_id = requests.get(target_url + '/get/2', proxies = proxies, headers = auth_header)
    if get_cust_id.status_code == 200:
        print("Get Customer by ID Response")
        print(get_cust_id.json())
        print()
    
    post = {'id': 2}
    fetch_customer_post = requests.post(target_url + '/fetch/customer', json = post, proxies = proxies, headers = auth_header)
    if fetch_customer_post.status_code == 200:
        print("Fetch Customer POST Response")
        print(fetch_customer_post.json())
        print()
    
    search = {'search': 'dleon'}
    search_customer_username = requests.post(target_url + '/search', json = search, proxies = proxies, headers = auth_header)
    if search_customer_username.status_code == 200:
        print("Search Customer POST Response")
        print(search_customer_username.json())
        print()
    

Get Customer by ID Response
{'cc_num': '3088793192304391', 'email': 'brittanyhill@hotmail.com', 'firstname': 'Kara', 'id': 2, 'lastname': 'Kemp', 'username': 'martineznorma'}

Fetch Customer POST Response
{'cc_num': '3088793192304391', 'email': 'brittanyhill@hotmail.com', 'firstname': 'Kara', 'id': 2, 'lastname': 'Kemp', 'username': 'martineznorma'}

Search Customer POST Response
[['Elizabeth', 'Flores', 'dleon']]



Now that we have authenticated to the app. Let's run an Active Scan against the authenticated context of the app

In [14]:
# using ZAP's ascan object to start scanning, with the "Light" Policy. If you don't specify the policy
# ZAP Automatically uses the "Default" policy
active_scan_id = zap.ascan.scan(target_url, scanpolicyname='Light')

print("active scan id: {0}".format(active_scan_id))

#now we can start monitoring the spider's status
while int(zap.ascan.status(active_scan_id)) < 100:
    print("Current Status of ZAP Active Scan: {0}%".format(zap.ascan.status(active_scan_id)))
    time.sleep(10)

active scan id: 0
Current Status of ZAP Active Scan: 1%
Current Status of ZAP Active Scan: 60%


### Some Observations

Clearly, you will see that ZAP found some security issues. However, if you look at the source code of our vulnerable Flask Application, you will realize that we might have missed some key vulnerabilities here: 

Source Code: https://github.com/we45/Vulnerable-Flask-App

This brings us to the some important questions about scaling security testing within our organization. 

- Yes, there will always be vulnerabilities that DAST tools cannot find. 
- These tools are limited by standard rules that are meant to identify vulnerabilities

So, with this....

- Can we create customized scripts for vulnerabilities that we might find in our Stack? 
    - Think like a DAST Linter for Apps
- Can we make it scale across our population of apps?

## Introducing ZAP's Scripting Framework

#### Supported Platforms
- Python (Jython)
- Ruby
- Oracle Nashorn ECMAScript
- Zest Scripts

#### Types of Scripts
- Active Rules => Scripts invoked during Active Scan
- Authentication Scripts => Scripts invoked to facilitate authentication for a Context
- Fuzzer Processors => Scripts invoked after Fuzzers are run with ZAP
- HTTPSender => Scripts invoked against every request/response received by ZAP
- Proxy => Runs inline and acts on all requests and responses
- Targeted Rules => Invoked on specific urls or on manual start only
- Standalone => Invoked manually
- Passive Rules => Passive Scanning Rules

#### Running Python Scripts with ZAP
- ZAP supports scripts written in Jython
- Python on Java JVM
- Not fully compatible with python libraries
- some limitations on networking and i/o libraries in python
- Works when “Python Scripting” add-on is installed in OWASP ZAP. 
- Third Party Python Libs can be linked when refer to the jython site-packages directory

### Useful Modules - ZAP Scripting with Python (Jython)

```python
msg
    #the message object that is acted upon to parse/manipulate

msg.getRequestHeader()
    #Request Header Object

msg.getRequestHeader().getURI()
    #fetches the URI from the request header

msg.getRequestBody()
    #Fetches the request body from the request

msg.getResponseBody()
    #Fetches the request body from the request

msg.setRequestBody()
    #Sets a different request body from the one in the original request
```

### Let's start with something simple...

Let's just quickly write a proxy script for ZAP and run it. If you are not too comfortable with the code, you can just copy-paste it. 

Go to ZAP GUI and click on the Scripts "+" Button: 
![Scripts Button](images/script_button.jpg)

Now, click on "Scripts". You should see this screen in the side-panel
![Scripts Panel](images/script_types.jpg)

Now, right click on "Proxy" and click on "New Script". Fill in the options as shown below:
![Scripts Panel](images/script_proxy_options.jpg)

Now, you can copy the code from below and paste it in the script code panel

```python
"""
The proxyRequest and proxyResponse functions will be called for all requests  and responses made via ZAP, 
excluding some of the automated tools
If they return 'false' then the corresponding request / response will be dropped. 
You can use msg.setForceIntercept(true) in either method to force a break point

Note that new proxy scripts will initially be disabled
Right click the script in the Scripts tree and select "enable"  
"""

def proxyRequest(msg):
  # Let's add a Custom Header to every request
  msg.getRequestHeader().setHeader('My-Custom-Header', "ZAPisAwesome");
  return True;

def proxyResponse(msg):
  return True;

```

Now you can save the script by right clicking on the script in the Script Panel and clicking on "Save"
![save script](save_script.jpg)

to check if this is working, just run the parameterization from above, once again...
![Custom Proxy Proof of Life](custom_proxy_poc.jpg)

### Alright, That was cute...So what can it really do?

Let's write some active scan rules to identify some real world vulnerabilities.

#### 1. The `none` signed JWT
Did you know that JWTs could be signed with the algorithm `none`, essentially meaning that they werent signed at all?? Crazy right? We've seen some implementations in the real world, where JWTs have been signed with the none algo, which means that an attacker can literally send any token he/she wants and get access to an authenticated view of the application. 

Here's how you setup a custom script for "Active Scan" rules. 
- Right click on "Active Scan" and select the "New Script" option
- Provide script options as follows: 
![None JWT Token](images/none_jwt_options.jpg)

You can copy the script from below and paste into the code panel

```python
alert_name = "Application is configured to accept 'none' signed JSON Web Tokens"
alert_desc = "Attacker can create fake tokens to authenticate to the application, using the 'none' signature. The application seems to be authenticate the user based on these 'none' signed tokens. Attackers can use this to bypass authentication and gain deeper access to the application."
alert_cwe = 287
alert_wasc = 1
alert_soln = 'Ensure that JSON Web Tokens are decoded and verified before authenticating and authorizing the user to perform actions on the application'
alert_risk = 3
alert_confidence = 1

dummy_jwt = 'eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpc3MiOiJodHRwczovL2p3dC1pZHAuZXhhbXBsZS5jb20iLCJzdWIiOiJtYWlsdG86bWlrZUBleGFtcGxlLmNvbSIsIm5iZiI6MTQ5OTg2MDkwNiwiZXhwIjoxNDk5ODY0NTA2LCJpYXQiOjE0OTk4NjA5MDYsImp0aSI6ImlkMTIzNDU2IiwidHlwIjoiaHR0cHM6Ly9leGFtcGxlLmNvbS9yZWdpc3RlciJ9.'

def scanNode(sas, msg):
    orig_msg = msg
    msg = orig_msg.cloneRequest()
    jwt_segments = dummy_jwt.split('.')
    header = jwt_segments[0]
    payload = jwt_segments[1]
    signature = jwt_segments[2]
    if orig_msg.getRequestHeader().getHeader('Authorization'):
        print "JWT: ", header, payload, signature
        msg.getRequestHeader().setHeader('Authorization', dummy_jwt)
        sas.sendAndReceive(msg, False, False)
        status_code = msg.getResponseHeader().getStatusCode()
        if status_code == 200:
            sas.raiseAlert(alert_risk, alert_confidence, alert_name, alert_desc,
                           msg.getRequestHeader().getURI().toString(), dummy_jwt, '', '',
                           alert_soln, '',
                           alert_cwe, alert_wasc, msg)




def scan(sas, msg, param, value):
    pass
```

Now, save script as you did before. Let's run an active scan!

If everything went well, ZAP should have identified another flaw. Your alert screen should look like this:
![zap alert](images/none_jwt_vul.jpg)

#### Wow! Now that it worked, let's try another script. 

Our Application uses the Jinja2 Templating System to render templates to the presentation layer. There may be a possibility of performing a Server-Side Template Injection against the app. Let's check it out with another OWASP ZAP Active Scan Rule

You can use the following options to setup the script and then "Save" it. 

You can copy-paste the following script into the code panel

```python
alert_name = "Jinja2 Server Side Template Injection"
alert_desc = "Server-Side Template Injection, where adversary can manipulate template variables. "
alert_cwe = 90
alert_wasc = 1
alert_soln = 'Jinja2 Server Side Template Injection'
alert_risk = 3
alert_confidence = 1

payload = '{{config.items()}}'

def scanNode(sas, msg):
    orig_msg = msg
    msg = orig_msg.cloneRequest()
    split_uri = str(msg.getRequestHeader().getURI()).split('/')
    if split_uri[:-1] is not None:
        mangle_var = split_uri[-1]
        del split_uri[-1]
        split_uri.append(payload)
        join_url = '/%s' % payload
        msg.getRequestHeader().getURI().setPath(join_url)
        print msg.getRequestHeader().getURI()
        sas.sendAndReceive(msg)
        print msg.getResponseBody()
        if 'JSON_AS_ASCII' in str(msg.getResponseBody()):
            sas.raiseAlert(alert_risk, alert_confidence, alert_name, alert_desc,
                           msg.getRequestHeader().getURI().toString(), payload, '', '',
                           alert_soln, '',
                           alert_cwe, alert_wasc, msg)





def scan(sas, msg, param, value):
    pass
```

Save it. Run another active scan with the "Light" Policy

If all's well, you should see a new vulnerability in your Alerts Panel with details of the vulnerability. 

![Server Side Template Injection](images/ssti_vul.jpg)